# Data Merging and Preprocessing

This notebook should only be run once to generate the final data in form of train and test data.

In [8]:
# load the df from csv
import pandas as pd
import numpy as np
import yfinance as yf
import json


# get date as datetime
from datetime import datetime, timedelta
from retrying import retry


df = pd.read_csv('../1. data/final/final_data.csv', index_col=0)

# suppress warnings, especially for FutureWarning passing literal json
import warnings
warnings.filterwarnings("ignore")

/Users/judi/Documents/Jonas/Advanced-Analytics-and-Machine-Learning/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


This way we can get the financial data in $n$ days from the publication to test our hypothesis

In [9]:
# make a def out of this
# we need two dates because its not always trading days
def get_dates(date):
    n = 120
    date = datetime.strptime(date, '%d/%m/%Y')
    date1 = date - timedelta(days=n)
    date1 = date1.strftime('%Y-%m-%d')
    date2 = date + timedelta(days=n)
    date2 = date2.strftime('%Y-%m-%d')
    return date1, date2


@retry(stop_max_attempt_number=5, wait_fixed=1000)
def get_market_data(start, end, symbol):
    price = yf.download(symbol, start=start, end=end, period='1d')
    return price

In [10]:


# loop over all rows and get dates with get_dates and then get data with get_market_data
# then save to csv with the ticker symbol as name
# also save in JSON file where the key is the ticker symbol and the values are the columns of the df
# for each column the key is the date and the value is the value of the column
# this way we can easily load the data for each ticker symbol
all_data = {}
for i in range(len(df)):
    # we split the date into day, month and year
    date = df["PriceDate"][i]
    start, end = get_dates(date)
    # format as YYYY-MM-DD with datetime
    symbol = df["FinalTickerSymbol"][i]
    try:
        data = get_market_data(start, end, symbol)
        data.to_csv(f"../1. data/raw/{df['FinalTickerSymbol'][i]}.csv")
        # get the price for the ticker symbol

        # Convert data to DataFrame
        data_df = pd.DataFrame(data)
        # Convert the index to a datetime
        data_df.index = pd.to_datetime(data_df.index)
        # Convert the columns to numeric values
        for col in data_df.columns:
            data_df[col] = pd.to_numeric(data_df[col], errors='coerce')

        # Convert DataFrame to JSON
        data_json = data_df.to_json(orient='columns')
        # Add data to all_data dictionary
        all_data[symbol] = data_json

    except:
        pass
    # print progress
    #print(f"{i+1}/{len(df)}")

with open('../1. data/raw/all_data.json', 'w') as f:
    json.dump(all_data, f)


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [11]:
df = pd.read_csv('../1. data/final/final_data.csv', index_col=0)
df = df[df["IsClose"] == True]
# then reset index
df = df.reset_index(drop=True)


with open('../1. data/raw/all_data.json') as f:
    all_data = json.load(f)

all_data_df = {key: pd.read_json(value, orient='columns') for key, value in all_data.items()}

#test 
all_data_df['VMUK.L'].head(2)



,Open,High,Low,Close,Adj Close,Volume
2023-07-26,172.199997,175.500000,172.199997,175.500000,173.27356,1713038
2023-07-27,175.649994,177.899994,173.800003,173.949997,171.74321,2250086


In [12]:


#the prediction is for a longer time horizon, so we can be less precise (6%)
def isclose(a, b, rel_tol=6e-02, abs_tol=0.0):
    return abs(a-b) <= max(rel_tol * max(abs(a), abs(b)), abs_tol)

# we loop over i elements in the df and check whether the price is within 5% of the price in the Price column
prediction_list = []
value = False
for i in range(len(df)):
    value = isclose(df["FairPrice"][i], df["Price"][i])
    if value == True:
        prediction_list.append(0)
    else:
        if df["FairPrice"][i] > df["Price"][i]:
            prediction_list.append(1)
        else:
            prediction_list.append(0)
    # print progress
    #print(f"{i+1}/{len(df)}")

df["Prediction"] = prediction_list


df["120dAvgPriceBefore"] = np.nan
df["120dAvgPriceAfter"] = np.nan

#the actual is for a shorter time horizon, so we have to be more precise (2%)
def isclose(a, b, rel_tol=2e-02, abs_tol=0.0):
    return abs(a-b) <= max(rel_tol * max(abs(a), abs(b)), abs_tol)

for i in range(len(df)):
    symbol = df["FinalTickerSymbol"][i]
    date = df["PriceDate"][i]
    # turn date into datetime
    date = datetime.strptime(date, '%d/%m/%Y')
    date = date.strftime('%Y-%m-%d')
    try:
        df["120dAvgPriceBefore"][i] = all_data_df[symbol]["Adj Close"][:date].mean()
        df["120dAvgPriceAfter"][i] = all_data_df[symbol]["Adj Close"][date:].mean()
    except:
        pass
    # print progress
    #print(f"{i+1}/{len(df)}")

trend_list = []
value = False
for i in range(len(df)):
    value = isclose(df["120dAvgPriceAfter"][i], df["TickerPrice"][i])
    if value == True:
        trend_list.append(0)
    else:
        if df["TickerPrice"][i] < df["120dAvgPriceAfter"][i]:
            trend_list.append(1)
        else:
            trend_list.append(0)
    # print progress
    #print(f"{i+1}/{len(df)}")

df["Label"] = trend_list


Now split in test and train

In [13]:
len(df)

1586

In [15]:
# count where "Label" is Nan
df["AnalystNoteList"].isna().sum()

197

In [16]:
#impute the missing values with "No Text Available"
df["AnalystNoteList"] = df["AnalystNoteList"].fillna("No Text Available")

In [17]:
from sklearn.model_selection import train_test_split

# Split the DataFrame into training and testing sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Save the training and testing sets as CSV files
test_df.to_csv('../1. data/final/test.csv', index=False)

train_df, dev_df = train_test_split(train_df, test_size=0.13, random_state=42)
train_df.to_csv('../1. data/final/train.csv', index=False)
dev_df.to_csv('../1. data/final/dev.csv', index=False)

In [18]:

len(train_df), len(dev_df), len(test_df)

(1103, 165, 318)